## Resumable Exceptions

Programming languages have a choice how to implement exceptions. Most, like Python, offer a way to recover from failures, but not to resume the *original* code.

Let's imagine we want to serve 2 customers, who want a beer and a wine:

In [1]:
from time import sleep
EmptyKegError = ValueError
def start_pouring(): pass
def stop_pouring(): pass

In [2]:
def serve_customer(desire):
    print(f"Hi, here is your {desire}:")
    try:
        if (desire == "beer"):
            provide_beer()
        else:
            print("🍷")
    except EmptyKegError:
        print("Sorry, the keg is empty.")
    print("Bye!\n")

def provide_beer():
    pour_beer()
    give_beer()

def pour_beer():
    start_pouring()
    while not glass_is_full:
        sleep(1)
        if keg_is_empty:
            raise EmptyKegError()
    stop_pouring()

def give_beer():
    print("🍺")

glass_is_full = False
keg_is_empty = True
serve_customer("beer")
serve_customer("wine")

Hi, here is your beer:
Sorry, the keg is empty.
Bye!

Hi, here is your wine:
🍷
Bye!



While we didn't satisfy the first customer, we recovered from the exception by saying "Bye!" and serving the next one.

Notice that we cannot resume the original code after an exception, we missed our chance to `give_beer()`. Sure, we can try to `provide_beer()` multiple times or `ask_for_something_else()`, but Python unwinds the call stack. While `traceback` has a method `traceback.print_stack()`, there is no `traceback.resume_as_if_nothing_happened()`.

Other languages might provide more advanced exception handling, such as [Common Lisp](https://en.wikibooks.org/wiki/Common_Lisp/Advanced_topics/Condition_System). Let's imagine "Python 4" would support following made-up syntax:

<!-- Following is a copy&pasted HTML generated by the above snippet, modified to mark handle/resume in bold text: -->

<pre><code class="cm-s-ipython language-python"><span class="cm-keyword">def</span> <span class="cm-def">serve_customer</span>(<span class="cm-variable">desire</span>):
    <span class="cm-builtin">print</span>(<span class="cm-string">f"Hi, here is your </span>{<span class="cm-variable">desire</span>}<span class="cm-string">:"</span>)
    <span class="cm-keyword">try</span>:
        <span class="cm-keyword">if</span> (<span class="cm-variable">desire</span> <span class="cm-operator">==</span> <span class="cm-string">"beer"</span>):
            <span class="cm-variable">provide_beer</span>()
        <span class="cm-keyword">else</span>:
            <span class="cm-builtin">print</span>(<span class="cm-string">"🍷"</span>)
    <span class="cm-strong">handle</span> <span class="cm-variable">EmptyKegError</span>:
        <span class="cm-builtin">print</span>(<span class="cm-string">"One second please, I have to change the keg."</span>)
        <span class="cm-strong">resume</span>
    <span class="cm-builtin">print</span>(<span class="cm-string">"Bye!\n"</span>)</code></pre>
---
<pre>
Hi, here is your beer:
One second please, I have to change the keg.
🍺
Bye!

Hi, here is your wine:
🍷
Bye!
</pre>

Wouldn't that be nice for the first customer?

Please notice how `provide_beer` is a simple function that can express our business logic. It does not have to care about all the things that can go wrong nor what to do if it does. We can decouple business logic from [Handling Exceptions](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).

## Algebraic Effects

In Python, exceptions aren't used for normal [Control Flow](https://docs.python.org/3/tutorial/controlflow.html) very often. It's easy to forget something with exceptions, have you noticed I did not `stop_pouring()` in my first example? We have other tools, like built-in methods:

In [3]:
# ✗ ugly
try:
    value = {"a": 1}["b"]
except KeyError:
    value = "default"

# ✓ better
value = {"a": 1}.get("b", "default")
value

'default'

Or the [with](https://docs.python.org/3/reference/compound_stmts.html#with) statement:

In [4]:
# ✗ ugly
try:
    file = open("out.txt", "w")
    file.write("Hello, world!")
finally:
    file.close()

# ✓ better
with open("out.txt", "w") as file:
    file.write("Hello, world!")

Also, Python supports side effect inside functions:

In [5]:
def greet():
    print("Hi!")

greet()

Hi!


Here, `greet` is not a pure function - it does not take an input, does not return an output and it modifies something in the world outside.

In contrast, functional languages like Haskell support only pure funtions. They provide built-in primitives for side effects, like `putStrLn "Hello, world"`, but to combine functions with side effects, we have to square a circle. Those languages introduce complicated concepts like [Monads](https://en.wikibooks.org/wiki/Haskell/Understanding_monads/IO). I do not understand Monads, so I won't provide any more insights. Monads seem to be so complicated, in fact, that some functional languages want to opt out - such as the [Eff](https://www.eff-lang.org/) language. The alternative is a concept very similar to the resumable exceptions, but used to control flow - [**Algebraic Effects**](https://overreacted.io/algebraic-effects-for-the-rest-of-us/).

While we don't have algebraic effects in Python 3, we can simulate them with generators:

In [6]:
currentHandlers = []

def withPerform(tryBlock, handle):
    global currentHandleIterator
    currentHandlers.append(handle)
    tryBlock()
    currentHandlers.pop()

def perform(effect):
    for generator in reversed(currentHandlers):
        try:
            iterator = generator()
            next(iterator)
            return iterator.send(effect)
        except StopIteration or TypeError:
            continue
            
import time
def sleep(s):
    time.sleep(s)
    global glass_is_full
    glass_is_full = True

In [7]:
def serve_customer(desire):
    print(f"Hi, here is your {desire}:")
    def tryBlock():
        if (desire == "beer"):
            provide_beer()
        else:
            print("🍷")
    def handle():
        effect = yield
        if (effect == "replace_keg"):
            print("One second please, I have to change the keg.")
            sleep(1)
            global keg_is_empty
            keg_is_empty = False
            yield "Now replaced!"
    withPerform(
        tryBlock,
        handle
    )
    print("Bye!\n")

def provide_beer():
    pour_beer()
    give_beer()

def pour_beer():
    start_pouring()
    while not glass_is_full:
        sleep(1)
        if keg_is_empty:
            result = perform("replace_keg")
            print(result)
    stop_pouring()

def give_beer():
    print("🍺")

glass_is_full = False
keg_is_empty = True
serve_customer("beer")
serve_customer("wine")

Hi, here is your beer:
One second please, I have to change the keg.
Now replaced!
🍺
Bye!

Hi, here is your wine:
🍷
Bye!



So the "effect" part of Algebraic Effects stands for performing side effects. And we do it in a way that is transparent for intermediate functions. Just like `provide_beer` does not now how to handle exceptions like `EmptyKegError` and built-in side effects like `print`, it does not have to know about user-defined side effects like `perform("replace_keg")` either.

Imagine `1 + (2 + 3) == (1 + 2) + 3`, where algebra provides a set of rules for numbers. So in functional languages, algebra provides a different set of rules for function composition. That is the meaning of "algebraic" - you can combine functions that know how to handle some effects with functions that know how to handle other effects: 

In [8]:
def main():
    print(perform('a'), end=", ")
    print(perform('b'))
    
def handleA():
    effect = yield
    if (effect == 'a'):
        yield 'A'

def handleB():
    effect = yield
    if (effect == 'b'):
        yield 'B'

def handleC():
    yield
    yield 'C'


withPerform(main, handleA)
withPerform(main, handleB)
withPerform(main, handleC)
withPerform(lambda: withPerform(main, handleA), handleB)

A, None
None, B
C, C
A, B


Does this looks similar to the Dependency Injection pattern? Just more dangerous because it's not explicit? Yes, I think so too. This concept is intended for languages with static types, which could infer the type of the intermediate functions (it would tell you that calling `provide_beer()` is unsafe unless you provide a handler for the "replace_keg" effect).

So, please don't use this pattern in your production code yet. This was just an introduction to Algebraic Effects, or Resumable Exceptions, for Python developers.

I wanted to present this topic on the [PyVo meetup](https://pyvo.cz/praha-pyvo/), but I had to `except TravelOnTheDayOfMeetupError: writeThisBlog()`, so your questions and ideas for improvement are welcome as GitHub Issues ¯\\\_(ツ)_/¯

I was inspired by [Algebraic Effects for the Rest of Us](https://overreacted.io/algebraic-effects-for-the-rest-of-us/), and Dan is a much better writer than me. He is just using JavaScript examples, so I tried to introduce this topic to Python developers too.